In [1]:
pip install beautifulsoup4


     |████████████████████████████████| 122kB 22.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

     |████████████████████████████████| 5.5MB 6.6MB/s eta 0:00:01     |███████████▉                    | 2.0MB 6.6MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
import numpy as np 
import pandas as pd

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#opening connection grabbing page
results = requests.get(url)
#parsing
soup = bs.BeautifulSoup(results.content,'lxml')
#grabbing only the 'table' portion of the page (right click inspect)
table = soup.find_all('table')[0]


In [6]:
df = pd.read_html(str(table))
#convert df to json string, orientation 'records':list like
data = pd.read_json(df[0].to_json(orient='records'))

In [7]:
data.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
#selecting only where Borough is filled
remove_nan = data[data['Borough'] != 'Not assigned']
remove_nan.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
org_data = remove_nan.groupby(["Postal Code","Borough"], as_index=False).agg(','.join)
org_data.head()


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:

#Replace 'Not Assigned' values in Neighborhood column by the Borough value
#-----------------np.where(condition, output when condition = true, output when condition = false)-----------------------
org_data['Neighborhood'] = np.where(org_data['Neighborhood'] == 'Not Assigned', org_data['Borough'], org_data['Neighborhood'])
                            

In [11]:
org_data.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
org_data.shape

(103, 3)

# Obtaining Lat and Lon and merging data sets

In [13]:
#geospatial data read
geo_url = "http://cocl.us/Geospatial_data"
geo_data = pd.read_csv(geo_url)

In [14]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
#merging data sets

data_merged = pd.merge(org_data, geo_data, on = 'Postal Code')

In [16]:
data_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Plotting neighborhoods in Scarborough on map

In [17]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge geopy --yes
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-2.9.2         |   py36h45558ae_0         613 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    pandas-1.0.5               |   py36h830a2c2_0        10.1 MB  conda-forge
    pysocks-1.7.1              |   py36h9f0ad1d_1          27 KB  conda-

In [20]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests  
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
!conda install -c conda-forge geopy --yes

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("All Required Libraries Imported!")

ModuleNotFoundError: No module named 'geocoder'

In [21]:

#-------------Unable to get coordinates from geolocator--------------
address = 'Scarborough, Canada'
geolocator = Nominatim(user_agent = "test")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough Canada are 43.773077, -79.257774.


In [189]:
map_Scarborough = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, nei in zip(data_merged['Latitude'], data_merged['Longitude'], data_merged['Neighborhood']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

In [24]:
CLIENT_ID = 'L4HUEDO3L43P2B4QZY02UGYFNJLRK5E4NIHWO3CBYSUHNXOD'

CLIENT_SECRET = 'KEAKC3Z4A1S3WCOWGIA3OC1LB1SXI00ALQZOJAFGB2IR5AME'

VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+CLIENT_SECRET)

Your credentails:
CLIENT_ID: L4HUEDO3L43P2B4QZY02UGYFNJLRK5E4NIHWO3CBYSUHNXOD
CLIENT_SECRET: KEAKC3Z4A1S3WCOWGIA3OC1LB1SXI00ALQZOJAFGB2IR5AME


In [25]:
radius = 700 
LIMIT = 100
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    43.773077, 
   -79.257774, 
    radius, 
   LIMIT)
results = requests.get(url2).json()

In [26]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.head()


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,...,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.neighborhood,venue.venuePage.id
0,e-0-5085ec39e4b0b1ead2eb0818-0,0,"[{'summary': 'This spot is popular', 'type': '...",5085ec39e4b0b1ead2eb0818,Disney Store,300 Borough Drive,in Scarborough Town Centre,43.775537,-79.256833,"[{'label': 'display', 'lat': 43.775537, 'lng':...",...,CA,Scarborough,ON,Canada,[300 Borough Drive (in Scarborough Town Centre...,"[{'id': '4bf58dd8d48988d1f3941735', 'name': 'T...",0,[],NaN,NaN
1,e-0-4c059bcd7083952134097bce-1,0,"[{'summary': 'This spot is popular', 'type': '...",4c059bcd7083952134097bce,SEPHORA,300 Borough Drive,at Scarborough Town Centre,43.775017,-79.258109,"[{'label': 'display', 'lat': 43.77501688366838...",...,CA,Scarborough,ON,Canada,[300 Borough Drive (at Scarborough Town Centre...,"[{'id': '4bf58dd8d48988d10c951735', 'name': 'C...",0,[],NaN,NaN
2,e-0-4c3c8e1f282203bb4ca3fcda-2,0,"[{'summary': 'This spot is popular', 'type': '...",4c3c8e1f282203bb4ca3fcda,American Eagle Store,300 Borough Drive,in Scarborough Town Centre,43.776012,-79.258334,"[{'label': 'display', 'lat': 43.7760116, 'lng'...",...,CA,Scarborough,ON,Canada,[300 Borough Drive (in Scarborough Town Centre...,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",0,[],NaN,NaN
3,e-0-4c648dbdf07e2d7fc4aa8f50-3,0,"[{'summary': 'This spot is popular', 'type': '...",4c648dbdf07e2d7fc4aa8f50,Hot Topic,300 Borough Dr.,at Scarborough Town Centre,43.775450,-79.257929,"[{'label': 'display', 'lat': 43.77545002191434...",...,CA,Scarborough,ON,Canada,"[300 Borough Dr. (at Scarborough Town Centre),...","[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",0,[],NaN,NaN
4,e-0-4c4bbc646e209521b57e16ec-4,0,"[{'summary': 'This spot is popular', 'type': '...",4c4bbc646e209521b57e16ec,DAVIDsTEA,300 Borough Dr Unit 253A,NaN,43.776320,-79.258688,"[{'label': 'display', 'lat': 43.77632000767913...",...,CA,Scarborough,ON,Canada,"[300 Borough Dr Unit 253A, Scarborough ON M1P ...","[{'id': '4bf58dd8d48988d1dc931735', 'name': 'T...",0,[],NaN,NaN


In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Store,Clothing Store,43.776012,-79.258334
3,Hot Topic,Clothing Store,43.775450,-79.257929
4,DAVIDsTEA,Tea Room,43.776320,-79.258688


In [29]:
# Top 10 cat
a=pd.Series(nearby_venues.categories)
a.value_counts()[:10]

Clothing Store            7
Coffee Shop               4
Restaurant                4
Furniture / Home Store    2
Gas Station               2
Tea Room                  2
Electronics Store         2
Pharmacy                  2
Sandwich Place            2
Deli / Bodega             1
Name: categories, dtype: int64

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # making GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
# Nearby Venues
Scarborough_venues = getNearbyVenues(names=data_merged['Neighborhood'],
                                   latitudes=data_merged['Latitude'],
                                   longitudes=data_merged['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [32]:
print('There are {} Uniques Categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

#number of venues in each neighbourhood
Scarborough_venues.groupby('Neighborhood').count().head()

There are 317 Uniques Categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,9,9,9,9,9,9
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Wilson Heights, Downsview North",24,24,24,24,24,24
Bayview Village,7,7,7,7,7,7
"Bedford Park, Lawrence Manor East",33,33,33,33,33,33


In [33]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(3411, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
2,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
3,"Malvern, Rouge",43.806686,-79.194353,Fortress Technology Inc.,43.801677,-79.194130,Business Service
4,"Malvern, Rouge",43.806686,-79.194353,Tim Hortons / Esso,43.801863,-79.199296,Coffee Shop


In [34]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]
Scarborough_grouped = Scarborough_onehot.groupby('Neighborhood').mean().reset_index()
Scarborough_onehot.head(5)

,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
num_top_venues = 5
for hood in Scarborough_grouped['Neighborhood']:
    print("---- "+hood+" ----")
    temp =Scarborough_grouped[Scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Agincourt ----
             venue  freq
0        Pool Hall  0.11
1           Lounge  0.11
2   Sandwich Place  0.11
3   Breakfast Spot  0.11
4  Motorcycle Shop  0.11


---- Alderwood, Long Branch ----
               venue  freq
0        Pizza Place   0.2
1  Convenience Store   0.2
2        Gas Station   0.1
3       Skating Rink   0.1
4     Sandwich Place   0.1


---- Bathurst Manor, Wilson Heights, Downsview North ----
              venue  freq
0       Coffee Shop  0.08
1              Bank  0.08
2              Park  0.08
3  Community Center  0.04
4        Restaurant  0.04


---- Bayview Village ----
                 venue  freq
0                 Bank  0.29
1                 Café  0.14
2        Grocery Store  0.14
3  Japanese Restaurant  0.14
4   Chinese Restaurant  0.14


---- Bedford Park, Lawrence Manor East ----
                venue  freq
0  Italian Restaurant  0.12
1         Coffee Shop  0.09
2          Restaurant  0.06
3      Sandwich Place  0.06
4              Bakery  0.03



In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Most common venues near neighbourhoods**

In [141]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarborough_grouped['Neighborhood']

# shape[0] refers to rows in Scarborough_grouped
for ind in np.arange(Scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Badminton Court,Clothing Store,Pool Hall,Breakfast Spot,Lounge,Motorcycle Shop,Sandwich Place,Latin American Restaurant,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Convenience Store,Gas Station,Pub,Sandwich Place,Skating Rink,Coffee Shop,Gym,Grocery Store,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Park,Bank,Shopping Mall,Fried Chicken Joint,Sushi Restaurant,Supermarket,Pharmacy,Bridal Shop,Middle Eastern Restaurant
3,Bayview Village,Bank,Japanese Restaurant,Café,Grocery Store,Skating Rink,Chinese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Restaurant,Breakfast Spot,Bagel Shop,Bakery,Sushi Restaurant,Bank,Juice Bar


**Trimming dataframe for only neighborhoods in Scarborough**

In [157]:
Scarborough_merged = data_merged.iloc[:16,:]
Scarborough_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.030303,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
Scarborough_merged = Scarborough_merged.join(Scarborough_grouped.set_index('Neighborhood'), on='Neighborhood')
Scarborough_merged.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [159]:
Scarborough_grouped_clustering = Scarborough_merged.drop(['Neighborhood','Postal Code', 'Borough', 'Cluster', 'Latitude', 'Longitude'], 1)
Scarborough_grouped_clustering.head()


,Yoga Studio,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**K-Means Clustering within Scarborough**

In [160]:
# set number of clusters
kclusters = 3
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [175]:
Scarborough_merged.insert(0, 'Cluster Labels', kmeans.labels_)

Scarborough_Nei = Scarborough_merged[['Neighborhood','Cluster Labels']].copy()
Scarborough_Ranked = Scarborough_Nei.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Scarborough_Ranked.head()


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",0,Fast Food Restaurant,Coffee Shop,Spa,Business Service,Donut Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run
1,"Rouge Hill, Port Union, Highland Creek",2,Breakfast Spot,Bar,Burger Joint,Dumpling Restaurant,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Guildwood, Morningside, West Hill",1,Park,Fast Food Restaurant,Restaurant,Intersection,Rental Car Location,Electronics Store,Breakfast Spot,Mexican Restaurant,Pharmacy,Sports Bar
3,Woburn,1,Park,Coffee Shop,Business Service,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
4,Cedarbrae,1,Indian Restaurant,Coffee Shop,Bakery,Bank,Fried Chicken Joint,Hakka Restaurant,Gym / Fitness Center,Athletics & Sports,Thai Restaurant,Gas Station


**Plotting Cluster**

In [190]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


clusters = kmeans.labels_
colors = ['red', 'green', 'blue','yellow']
Scarborough_merged['Cluster'] = clusters

for latitude, longitude, nei, cluster in zip(Scarborough_merged['Latitude'],
                                             Scarborough_merged['Longitude'],
                                             Scarborough_merged['Neighborhood'],
                                             Scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)  

map_clusters

In [184]:

df1=Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 0]
df2=Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 1]
df3=Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 2]

In [185]:
clusters=pd.DataFrame({"Cluster1":df1["Neighborhood"],
                      "Cluster2":df2["Neighborhood"],
                      "Cluster3":df3["Neighborhood"]})
clusters = clusters.replace(np.nan, '', regex=True)

In [186]:
clusters

,Cluster1,Cluster2,Cluster3
0,"Malvern, Rouge",,
1,,,"Rouge Hill, Port Union, Highland Creek"
2,,"Guildwood, Morningside, West Hill",
3,,Woburn,
4,,Cedarbrae,
5,,Scarborough Village,
6,,"Kennedy Park, Ionview, East Birchmount Park",
7,,"Golden Mile, Clairlea, Oakridge",
8,,"Cliffside, Cliffcrest, Scarborough Village West",
9,,"Birch Cliff, Cliffside West",


**For this project, I have sliced Scarborough from the dataframe of the state of toronto and applied K-Means clustering based on the catogories of the venues in each neighbourhood within scarborough.**